#Atividade 3 - Programação Matemática
##Alunos
 - Daniel Messias Santos - 202110168
 - Thiago Pereira Freire - 202110167

#Enunciado

A cidade de União Paulista enfrenta uma séria carência orçamentária. Em busca de uma solução de longo prazo, a câmara de vereadores da cidade aprova uma melhoria da base da cobrança de impostos que prevê a condenação de uma área habitacional no centro da cidade e sua substituição por um conjunto habitacional moderno.

O projeto envolve duas fases:

1. Demolição das casas que estão aquém do padrão para liberar terreno para o novo projeto.
2. Construção do novo conjunto urbano.

Um total de 300 casas aquém do padrão podem ser demolidas. Cada casa ocupa um lote de 0,25 acres. O custo de demolição de uma casa condenada é de $2.000. Os tamanhos dos lotes para domicílios simples, duplos, triplos e quádruplos são de 0,18; 0,28; 0,4 e 0,5 acres, respectivamente. Ruas, espaços abertos e instalações públicas ocupam 15% da área disponível.

No novo conjunto habitacional, as unidades triplas e quádruplas devem representar, no mínimo, 25% do total. Unidades simples devem representar, no mínimo, 20% de todas as unidades, e unidades duplas, no mínimo, 10%. O imposto cobrado por unidade para unidades simples, duplas, triplas e quádruplas é de \$1.000, \$1.900, \$2.700 e \$3.400, respectivamente.

O custo de construção por unidade domiciliar simples, dupla, tripla e quádrupla é de \$50.000, \$70.000, \$130.000 e \$160.000, respectivamente. O financiamento acordado com um banco local será de, no máximo, \$15 milhões.

Quantas unidades de cada tipo devem ser construídas para maximizar a arrecadação de impostos?

## Parâmetros
| Tipo de casa | Tamanho     | Imposto | Custo de produção|
|--------------|-------------|---------|------------------|
| simples      | 0.18 acres  | 1000    |50.000            |
| duplas       | 0.28 acres  | 1900    |70.000            |
| triplas      | 0.40 acres  | 2700    |130.000           |
| quádruplas   | 0.50 acres  | 3400    |160.000           |

- Custo de demolição por casa: 2.000
- Tamanho de cada casa condenada: 0.25 acres

## Variáveis
- Quantidade de casas simples X
- Quantidade de casas duplas Y
- Quantidade de casas triplas Z
- Quantidade de casas quádruplas W
- Quantidade de casas demolidas V

## Restrições
- Casas simples devem ocupar pelo menos 20% da quantidade de unidades
\begin{equation}
    (w + x + y + z)0,2 \le X
    \end{equation}
- Casas duplas devem ocupar pelo menos 10% da quantidade de unidades
\begin{equation}
    (w + x + y + z)0,1 \le Y
\end{equation}
- Casas triplas devem ocupar pelo menos 25% da quantidade de unidades
\begin{equation}
    (w + x + y + z)0,25 \le Z
\end{equation}
- Casas quádruplas devem ocupar pelo menos 25% da quantidade de unidades
\begin{equation}
    (w + x + y + z)0,25 \le W
\end{equation}
- O custo máximo é de 15.000.000
\begin{equation}
    50.000X + 70.000Y + 130.000Z + 160.000W + 2000V \le 15.000.000
\end{equation}
- Ruas ocupam 15% da área total
\begin{equation}
    0.18X + 0.28Y + 0.40Z + 0.50W \le 0.25V * 0.85
\end{equation}
- Existem 300 casas condenadas no local
\begin{equation}
    V \le 300
\end{equation}

## Função objetivo
\begin{equation}
    max f(X,Y,Z,W) = 1000X + 1900Y + 2700Z + 3400W
\end{equation}

### Instalação da biblioteca

In [21]:
!pip install pulp
import pulp

### Parâmetros

In [22]:
casas = [0, 1, 2, 3]
tamanhos = (0.18, 0.28, 0.40, 0.50)
impostos = (1000, 1900, 2700, 3400)
custos = (50000, 70000, 130000, 160000)
custoDemolicao = 2000
tamanhoCondenada = 0.25
quantidadeMinima = (0.2, 0.1, 0.25, 0.25)
custoMaximo = 15000000

### Criação do modelo

In [23]:
modelo = pulp.LpProblem("Casa", pulp.LpMaximize)

### Variáveis de decisão

In [24]:
quantidadeCasas = pulp.LpVariable.dicts("Quantidade", casas, lowBound=0, cat='Integer')
quantidadeCondenadas = pulp.LpVariable("QuantidadeCondenada", lowBound=0, cat='Integer')

### Função objetivo

In [25]:
modelo += pulp.lpSum(quantidadeCasas[i] * impostos[i] for i in casas)

### Restrições

In [26]:
for i in casas:
  modelo += pulp.lpSum(quantidadeCasas[j] for j in casas) * quantidadeMinima[i] <= quantidadeCasas[i]
  modelo += quantidadeCasas[i] >= 0

modelo += pulp.lpSum(quantidadeCasas[i] * custos[i] for i in casas) + custoDemolicao * quantidadeCondenadas <= custoMaximo
modelo += quantidadeCondenadas >= 0
modelo += quantidadeCondenadas <= 300
modelo += pulp.lpSum(quantidadeCasas[i] * tamanhos[i] for i in casas) <= quantidadeCondenadas * tamanhoCondenada * 0.85

### Solver

In [27]:
status = modelo.solve()

### Impressão dos resultados

In [28]:
print(f"Status: {modelo.status}, {pulp.LpStatus[modelo.status]}")
total = 0
tiposResidencias = ["simples", "duplas", "triplas", "quádruplas"]
for i in casas:
  print(f"{tiposResidencias[i]}: quantidade: {quantidadeCasas[i].varValue}")
  total += quantidadeCasas[i].varValue * custos[i]
print(f"Quantidade de casas condenadas: {quantidadeCondenadas.varValue}")
print(f"Impostos arrecadados: {pulp.value(modelo.objective)}")
total += custoDemolicao * quantidadeCondenadas.varValue
print(f"Custo total: {total}")

Status: 1, Optimal
simples: quantidade: 28.0
duplas: quantidade: 42.0
triplas: quantidade: 35.0
quádruplas: quantidade: 35.0
Quantidade de casas condenadas: 228.0
Impostos arrecadados: 321300.0
Custo total: 14946000.0
